In [2]:
import openmm
from openmm import app, unit
from openmmml import MLPotential

In [3]:
import urllib
urllib.request.urlretrieve(
    "https://raw.githubusercontent.com/openmm/openmm/master/wrappers/python/tests/systems/alanine-dipeptide-explicit.pdb",
    "input.pdb"
)
pdb = app.PDBFile('input.pdb')

In [4]:
potential = MLPotential('ani2x')

In [5]:
system = potential.createSystem(pdb.topology)

/Users/wangy1/miniconda3/envs/openmm/lib/python3.11/site-packages/torchani/aev.py:16: UserWarning: cuaev not installed
  warnings.warn("cuaev not installed")
/Users/wangy1/miniconda3/envs/openmm/lib/python3.11/site-packages/torchani/__init__.py:55: UserWarning: Dependency not satisfied, torchani.ase will not be available
  warnings.warn("Dependency not satisfied, torchani.ase will not be available")


/Users/wangy1/miniconda3/envs/openmm/lib/python3.11/site-packages/torchani/resources/
failed to equip `nnpops` with error: No module named 'NNPOps'
